# Speculation Strategy
The following is a strategy to choose what stocks to buy. The theory of the strategy works off the assumptions that the stock market moves at a random walk, that prices return to their mean, and that the stock market is forever growing. There will be three stages where we start with a complete list of stocks and after each stage stocks will be eliminated until there is relatively small list of stocks to focus on. 

## Libraries needed

In [279]:
import pandas as pd
from datetime import date
import numpy as np
from pandas_datareader import data as web
from selenium import webdriver
import time

## Data
From [eooddata.com](https://eoddata.com/myaccount/default.aspx) I am able to import the closing prices from various stock markets. For this example we'll be using the closing prices of the New York Stock Exchange on the 21st and 28th of August of 2020.

In [137]:
colnames=['ticker','no','no_1','no_2','no_3','close','no_4'] #various columns will be ignored 
one=pd.read_csv('NYSE_20200821.txt',names=colnames,header=None)
two=pd.read_csv('NYSE_20200828.txt',names=colnames,header=None)
one=one[['ticker','close']]
two=two[['ticker','close']]
one=one.rename(columns={'close':'close_1'})
two=two.rename(columns={'close':'close_2'})

## Stocks prices that fell over the week (stage 1)
The first stage is to find stocks prices that fell over the past week. The idea being that these stocks could be good picks because of their lower price. I chose stocks that fell at least 2%. 

In [255]:
stocks=pd.merge(one,two,on='ticker')
stocks['percent_change']=((stocks.close_2-stocks.close_1)/stocks.close_1)*100
stocks=stocks[['ticker','percent_change']]
stocks=stocks[stocks.percent_change<-2] #stocks that fell 2%
stocks=stocks.reset_index(drop=True)
stocks['RSI']=0 #part of stage 2
stocks.head(2)

,ticker,percent_change,RSI
0,ACH,-4.654655,0
1,ACI,-3.624161,0


After the first stage we went from over 3 thousand tickers to 332.

In [256]:
print(len(one))
print(len(two))
print(len(stocks))

3089
3109
332


## Relative Strength Index (stage 2)
The Relative Strength Index (RSI) is a financial indicator that measures its current price behavior against a past period. I chose 14 days as a period. If a stock is sold at a higher rate than its previous period than it can be considered to be oversold and will have a low RSI score. [Yahoo Finance](https://finance.yahoo.com/) allows for remote acces to the closing prices of many different stocks. From these historical closing prices we can compute the RSI. 

In [262]:
start = date(2020,3,1)
for i,r in stocks.iterrows():
    name=r[0]
    try:
        ticker=web.DataReader(name,'yahoo',start=start)
        delta=ticker['Adj Close'].diff(1)
        delta=delta.dropna()
        up=delta.copy()
        down=delta.copy()
        up[up<0]=0
        down[down>0]=0
        period=14
        AVG_Gain=up.rolling(window=period).mean()
        AVG_Loss=abs(down.rolling(window=period).mean())
        RS = AVG_Gain/AVG_Loss
        RSI=100.0-(100.0/(1.0+RS))
        stocks.iloc[i,2]=RSI[-1]
    except Exception as ex:
        print('Ex:', ex)

Ex: No data fetched for symbol ACP using YahooDailyReader
Ex: No data fetched for symbol AEE using YahooDailyReader
Ex: index out of bounds
Ex: No data fetched for symbol AHL-C using YahooDailyReader
Ex: No data fetched for symbol AHL-D using YahooDailyReader
Ex: No data fetched for symbol AHL-E using YahooDailyReader
Ex: No data fetched for symbol AHT-D using YahooDailyReader
Ex: No data fetched for symbol AHT-F using YahooDailyReader
Ex: No data fetched for symbol AHT-H using YahooDailyReader
Ex: No data fetched for symbol AHT-I using YahooDailyReader
Ex: No data fetched for symbol AIG.W using YahooDailyReader
Ex: No data fetched for symbol AKO.A using YahooDailyReader
Ex: No data fetched for symbol AKO.B using YahooDailyReader
Ex: No data fetched for symbol ALI-A using YahooDailyReader
Ex: No data fetched for symbol ALL-H using YahooDailyReader
Ex: No data fetched for symbol ALT.W using YahooDailyReader
Ex: No data fetched for symbol AMRX using YahooDailyReader
Ex: No data fetched f

## Ticker elimination
From various errors (printed above) while accessing Yahoo and only keeping stocks that have an RSI under 40 we reduced our list to 121 stocks. 

In [263]:
stocks=stocks[(stocks.RSI!=0.0) & (stocks.RSI<40)] 
len(stocks)

121

In [269]:
stocks=stocks.sort_values(by='RSI')
stocks['news']=0
stocks=stocks.reset_index(drop=True)
stocks.head(2)

,ticker,percent_change,RSI,news
0,VMC,-2.336486,9.535437,0
1,UTI,-2.035278,10.937434,0


MTL.P was removed because the stock was not found on the site used for stage 3

In [285]:
stocks=stocks[stocks.ticker != 'MTL.P']
stocks=stocks.reset_index(drop=True)
len(stocks)

120

## Market sentiment (stage 3)
One reason that a stock might have fell in price is because of bad news. We would want to avoid these stocks because they are unlikely to grow after bad news. [IBM](https://discovery-news-demo.ng.bluemix.net/) developed a resource that quantifies the sentiment of a given company. Through web automation I will capture the negative score of the 120 remaining stocks. 

In [286]:
web = webdriver.Chrome()
web.get('https://discovery-news-demo.ng.bluemix.net/')
for i,r in stocks.iterrows():
    ticker=r[0]
    insert_ticker=web.find_element_by_xpath('/html/body/div[1]/div/div[2]/section/div/div[1]/div/label/input')
    insert_ticker.send_keys(ticker)
    search_button=web.find_element_by_xpath('/html/body/div[1]/div/div[2]/section/div/div[1]/div/button')
    search_button.click()
    time.sleep(4)
    score=web.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/div/div[2]/div/div/div/div[2]/div[2]/div[1]/div[1]').text
    stocks.iloc[i,3]=score
    web.get('https://discovery-news-demo.ng.bluemix.net/')

In [295]:
stocks['news']=stocks['news'].str.rstrip('%').astype('int')
stocks.head(2)

,ticker,percent_change,RSI,news
0,VMC,-2.336486,9.535437,25
1,UTI,-2.035278,10.937434,43


## Combining the RSI and News 
After creating and sorting our new indicator 'score' we can identify companies that had a sudden drop in price while not having much negative news in the headlines. The next step would be to investigate these companies individually and decide if you want to invest in them. 

In [296]:
stocks['score']=stocks.RSI+stocks.news
stocks=stocks.sort_values(by='score')
stocks.head(20)

,ticker,percent_change,RSI,news,score
12,TRC,-2.169492,16.666636,8,24.666636
4,GTX,-50.598802,14.260871,12,26.260871
16,BSMX,-3.539823,19.672123,7,26.672123
9,JMIA,-4.900000,16.003789,13,29.003789
3,BFZ,-2.479942,13.124788,17,30.124788
48,LAIX,-12.547529,27.272725,6,33.272725
10,TPL,-3.786357,16.308633,17,33.308633
19,SWX,-4.309816,19.883772,14,33.883772
0,VMC,-2.336486,9.535437,25,34.535437
35,PACD,-3.297518,25.000003,13,38.000003


## Closing remarks
After trying this strategy for one month I was able to sell 20/34 of the stocks I bought with an average of return of 2.62%. However, the other 14 stocks fell so much that my overall average was -0.76%. Furtheremore, I wasted many hour monitoring my various bad investments. I do not recommend this strategy and at most it should be used as a supplement to a more successful strategy. 

Nevertheless, I'm proud that I was able nearly automate the whole process.    